In [1]:
!pwd

/Users/ashish1610dhiman/data_projects/fall22_hw/RoboChef/notebooks/recommendation


In [2]:
from sklearn.manifold import TSNE

%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns
import pandas as pd
import numpy as np
from surprise import accuracy, Dataset, SVD,SVDpp, NMF
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split as train_test_split_surprise
from tqdm.notebook import tqdm

import numpy as np
from sklearn.metrics import ndcg_score
import scipy.stats as ss

In [3]:
import sys
from os.path import dirname
sys.path.append(dirname("../../"))

from src.utilities import *
from src.reccomendation_util import *

In [4]:
%load_ext autoreload
%autoreload 2

### Load Model and Raw Data

In [5]:
raw_interactions = pd.read_csv("../../data/recommendation/RAW_interactions.csv")
print(raw_interactions.shape)

(1132367, 5)


In [6]:
recipe_metadata = pd.read_csv("../../data/recommendation/RAW_recipes.csv")
recipe_metadata.shape

(231637, 13)

In [7]:
### Load Models
model1 = load_pickle("../../models/reccomender_model1_svd.pkl")
model2 = load_pickle("../../models/reccomender_model2_nmf.pkl")

In [8]:
model1.predict(uid=1533,iid=44459)

Prediction(uid=1533, iid=44459, r_ui=None, est=4.979205256134577, details={'was_impossible': False})

In [9]:
model1.test([[1533,57549,None],\
            [1533,44459,None],\
            [1533,31048,None],\
            [1533,10554,None]])

[Prediction(uid=1533, iid=57549, r_ui=None, est=4.984623436267778, details={'was_impossible': False}),
 Prediction(uid=1533, iid=44459, r_ui=None, est=4.979205256134577, details={'was_impossible': False}),
 Prediction(uid=1533, iid=31048, r_ui=None, est=4.909425973113856, details={'was_impossible': False}),
 Prediction(uid=1533, iid=10554, r_ui=None, est=4.904533254551073, details={'was_impossible': False})]

In [10]:
model2.test([[1533,57549,None],\
            [1533,44459,None],\
            [1533,31048,None],\
            [1533,10554,None]])

[Prediction(uid=1533, iid=57549, r_ui=None, est=5, details={'was_impossible': False}),
 Prediction(uid=1533, iid=44459, r_ui=None, est=5, details={'was_impossible': False}),
 Prediction(uid=1533, iid=31048, r_ui=None, est=5, details={'was_impossible': False}),
 Prediction(uid=1533, iid=10554, r_ui=None, est=5, details={'was_impossible': False})]

### Meta

In [11]:
recipe_metadata['nutrition_list'] = recipe_metadata['nutrition'].apply(lambda x: ast.literal_eval(x))
recipe_metadata[['calories','fat_dv',"sugar_dv",\
           "sodium_dv","protein_dv","sat_fat", "carbs_dv"]\
      ] = pd.DataFrame(recipe_metadata.nutrition_list.tolist(), index= recipe_metadata.index)

In [12]:
example_ingredients = ['apple juice', 'raw honey', 'whole cloves',\
                        'everclear alcohol', 'ground cinnamon', 'cheesecloth']

In [13]:
recipe_metadata['ingredients_list'] = recipe_metadata['ingredients'].apply(lambda x: ast.literal_eval(x))

In [14]:
def match_ingredients(x,to_match, threshold = 0.25):
    x = set(x)
    to_match = set(to_match)
    match_rate = len(x.intersection(to_match))/len(x)
    return match_rate>=threshold

In [15]:
recipe_metadata[recipe_metadata["ingredients_list"].apply(lambda x: match_ingredients(x,example_ingredients))]

,Unnamed: 0,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,...,n_ingredients,nutrition_list,calories,fat_dv,sugar_dv,sodium_dv,protein_dv,sat_fat,carbs_dv,ingredients_list
317,317,apple pie,124853,43200,213699,2005-06-06,"['time-to-make', 'course', 'main-ingredient', ...","[285.8, 0.0, 281.0, 0.0, 0.0, 0.0, 24.0]",2,['mix ingredence together except everclear and...,...,6,"[285.8, 0.0, 281.0, 0.0, 0.0, 0.0, 24.0]",285.8,0.0,281.0,0.0,0.0,0.0,24.0,"[apple juice, raw honey, whole cloves, evercle..."
820,820,10 minute applesauce alton brown,121344,20,66322,2005-05-10,"['30-minutes-or-less', 'time-to-make', 'course...","[270.1, 9.0, 189.0, 2.0, 1.0, 18.0, 19.0]",5,"['in a sealable microwave-safe container , com...",...,7,"[270.1, 9.0, 189.0, 2.0, 1.0, 18.0, 19.0]",270.1,9.0,189.0,2.0,1.0,18.0,19.0,"[golden delicious apples, fuji apples, apple j..."
2282,2282,absolut vince vega,261358,5,327115,2007-10-25,"['15-minutes-or-less', 'time-to-make', 'course...","[65.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",2,"['build over ice in a highball glass', 'decora...",...,2,"[65.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",65.5,0.0,0.0,0.0,0.0,0.0,0.0,"[absolut citron vodka, apple juice]"
2634,2634,advieh,253004,5,292560,2007-09-14,"['15-minutes-or-less', 'time-to-make', 'course...","[98.2, 3.0, 1.0, 0.0, 5.0, 1.0, 7.0]",5,['whizz the rose petals in the spice grinder u...,...,4,"[98.2, 3.0, 1.0, 0.0, 5.0, 1.0, 7.0]",98.2,3.0,1.0,0.0,5.0,1.0,7.0,"[dried rose petals, ground cinnamon, ground ca..."
3555,3555,almond banana shake,340071,4,599450,2008-11-30,"['15-minutes-or-less', 'time-to-make', 'course...","[111.5, 0.0, 57.0, 0.0, 2.0, 0.0, 9.0]",2,"['add banana slices , almond milk , and cinnam...",...,3,"[111.5, 0.0, 57.0, 0.0, 2.0, 0.0, 9.0]",111.5,0.0,57.0,0.0,2.0,0.0,9.0,"[banana, almond milk, ground cinnamon]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227737,227737,winter spice how nice warm breakfast drink,275801,4,37305,2008-01-03,"['15-minutes-or-less', 'time-to-make', 'course...","[107.0, 1.0, 0.0, 6.0, 19.0, 2.0, 5.0]",2,"['place milk , carnation instant breakfast , g...",...,4,"[107.0, 1.0, 0.0, 6.0, 19.0, 2.0, 5.0]",107.0,1.0,0.0,6.0,19.0,2.0,5.0,"[skim milk, instant breakfast drink mix, groun..."
227739,227739,winter apple jelly,484618,35,2344388,2012-07-30,"['60-minutes-or-less', 'time-to-make', 'course...","[698.4, 0.0, 682.0, 0.0, 0.0, 0.0, 60.0]",12,"['in a dutch oven , combine the apple juice , ...",...,6,"[698.4, 0.0, 682.0, 0.0, 0.0, 0.0, 60.0]",698.4,0.0,682.0,0.0,0.0,0.0,60.0,"[apple juice, powdered fruit pectin, lemon jui..."
227932,227932,witches brew,37830,5,37636,2002-08-20,"['15-minutes-or-less', 'time-to-make', 'course...","[108.8, 0.0, 104.0, 0.0, 0.0, 0.0, 9.0]",4,"['combine all ingredients and chill', 'pour in...",...,4,"[108.8, 0.0, 104.0, 0.0, 0.0, 0.0, 9.0]",108.8,0.0,104.0,0.0,0.0,0.0,9.0,"[frozen lemonade concentrate, apple juice, gin..."
229398,229398,yogurt cheese,56555,5760,10404,2003-03-18,"['weeknight', 'time-to-make', 'course', 'main-...","[254.2, 1.0, 139.0, 14.0, 52.0, 2.0, 11.0]",7,['set a strainer lined with 2 layers of cheese...,...,2,"[254.2, 1.0, 139.0, 14.0, 52.0, 2.0, 11.0]",254.2,1.0,139.0,14.0,52.0,2.0,11.0,"[nonfat yogurt, cheesecloth]"


### Reccomendation Testting

In [16]:
gen_recipe_list(raw_interactions,1533)

TypeError: gen_recipe_list() missing 1 required positional argument: 'user_id'

In [ ]:
gen_recipe_list(raw_interactions,1533)

In [ ]:
get_reccomendation_ids(model1,1533,gen_recipe_list(raw_interactions,1533),20)

In [ ]:
rec_ids = get_reccomendation_ids(model1,1533,gen_recipe_list(raw_interactions,1533),20)

In [ ]:
recipe_meta_map(recipe_metadata,rec_ids)

### Test final function

In [ ]:
example_ingredients

In [ ]:
#Read data
raw_interactions = pd.read_csv("../../data/recommendation/RAW_interactions.csv")
recipe_metadata = pd.read_csv("../../data/recommendation/RAW_recipes.csv")
recipe_metadata['nutrition_list'] = recipe_metadata['nutrition'].apply(lambda x: ast.literal_eval(x))
recipe_metadata['ingredients_list'] = recipe_metadata['ingredients'].apply(lambda x: ast.literal_eval(x))
recipe_metadata[['calories', 'fat_dv', "sugar_dv", \
                     "sodium_dv", "protein_dv", "sat_fat", "carbs_dv"] \
               ] = pd.DataFrame(recipe_metadata.nutrition_list.tolist(), index=recipe_metadata.index)

In [ ]:
###Basic
ad_final_reccom(1533,example_ingredients)

In [ ]:
#Read data
raw_interactions = pd.read_csv("../../data/recommendation/RAW_interactions.csv")
recipe_metadata = pd.read_csv("../../data/recommendation/RAW_recipes.csv")
recipe_metadata['nutrition_list'] = recipe_metadata['nutrition'].apply(lambda x: ast.literal_eval(x))
recipe_metadata['ingredients_list'] = recipe_metadata['ingredients'].apply(lambda x: ast.literal_eval(x))
recipe_metadata[['calories', 'fat_dv', "sugar_dv", \
                     "sodium_dv", "protein_dv", "sat_fat", "carbs_dv"]\
               ] = pd.DataFrame(recipe_metadata.nutrition_list.tolist(), index=recipe_metadata.index)

In [ ]:
protein_mask = recipe_metadata["protein_dv"]>20.0

In [ ]:
###Basic
ad_final_reccom(1533,example_ingredients,raw_interactions, recipe_metadata)

In [ ]:
###With Protein mask
ad_final_reccom(1533,example_ingredients,raw_interactions, recipe_metadata,\
                extra_filters=[protein_mask])

In [ ]:
###With Protein mask
ad_final_reccom(1533,example_ingredients,raw_interactions, recipe_metadata,\
                extra_filters=[protein_mask],threshold=0.4)

In [ ]:
###With Protein mask
ad_final_reccom(1533,example_ingredients,raw_interactions, recipe_metadata,\
                extra_filters=[protein_mask],threshold=0.4)